In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
import bz2
import zipfile
import locale
from os import path

In [2]:
def get_replay(match_id):
    #check if replay downloaded
    
    
    url = "https://api.opendota.com/api/replays?match_id={}".format(match_id)
    replay_info = requests.get(url)
    
    #get salt and cluster from opendota
    url = "https://api.opendota.com/api/replays?match_id={}".format(match_id)
    raw_json = requests.get(url)
    replay_info = json.loads(raw_json.text)[0]
    
    #download from valve servers
    url = "http://replay{}.valve.net/570/{}_{}.dem.bz2".format(replay_info['cluster'], replay_info['match_id'], replay_info['replay_salt'])
    replay = requests.get(url, allow_redirects=True)
    #with open('{}.bz2'.format(match_id), 'wb') as f:
    #    f.write(replay.content)
    decompressed = bz2.decompress(replay.content)
    with open('replays/{}.dem'.format(replay_info['match_id']), 'wb') as f:
        f.write(decompressed)

In [3]:
def update():
    r = requests.get("http://dota2protracker.com")
    soup = BeautifulSoup(r.text)
    raw_links = soup.findAll('a', href=re.compile('dotabuff'))
    match_ids = []
    for link in raw_links:
        match_id = re.search(r"\d{10,}", str(link))[0]
        match_ids.append(match_id)
    for match_id in match_ids:
        if not path.exists(match_id):
            get_replay(match_id)

In [43]:
url = "https://api.opendota.com/api/matches/5432043477"
raw_json = requests.get(url)
api_info = json.loads(raw_json.text)

In [47]:
api_info['players'][0]['account_id']

In [18]:
url = "https://api.opendota.com/api/matches/5411957111"
raw_json = requests.get(url)
api_info = json.loads(raw_json.text)

In [36]:
for player in api_info['players']:
    print(player['player_slot'])


0
1
2
3
4
128
129
130
131
132


In [42]:
api_info['players'][0]['account_id']

118721929

In [21]:
print(json.dumps(api_info, indent=4))

{
    "match_id": 5411957111,
    "barracks_status_dire": 0,
    "barracks_status_radiant": 63,
    "chat": [
        {
            "time": -75,
            "type": "chatwheel",
            "key": "64",
            "slot": 9,
            "player_slot": 132
        },
        {
            "time": -75,
            "type": "chat",
            "unit": "general fun #1",
            "key": "\u043f",
            "slot": 1,
            "player_slot": 1
        },
        {
            "time": -23,
            "type": "chatwheel",
            "key": "46002",
            "slot": 5,
            "player_slot": 128
        },
        {
            "time": -17,
            "type": "chat",
            "unit": "through fire",
            "key": "mirana pls dont come mid",
            "slot": 3,
            "player_slot": 3
        },
        {
            "time": -10,
            "type": "chat",
            "unit": "Cuzz",
            "key": "block wit wolf =)",
            "slot": 5,
            "pl